# 作业一

**问题（look_at_cc）：4 分**

(a) 下载上面的 WARC 文件，或者找到我们在集群上提供的副本。让我们看看这个文件中的第一页。这是一个 gzipped 文件，你可以使用以下命令浏览其内容：

```
$ zcat /data/CC/example.warc.gz | less
```

`less` 让你可以使用键盘箭头、Page Up、Page Down 来浏览文件。要退出，按“q”。
查看非常第一个网页。它的 URL 是什么？它仍然可以访问吗？你能通过查看原始 HTML 来告诉我们这个页面似乎是什么吗？
交付物：2-3 句回应。

(b) 现在让我们看看对应的 WET 文件：

```
$ zcat /data/CC/example.warc.wet.gz | less
```

注意 WET 文件包含 HTTP 头（例如，Content-Length），这些不是提取的文本内容的一部分。如果你查看第一个例子，你会看到它包含从你刚刚看到的原始 HTML 中提取的文本。

注意提取的文本中有多少是 HTML 结构的遗迹，而不是页面的主要内容。你认为哪些部分应该被提取器过滤掉？将此文本作为训练数据时，可能会出错的地方是什么：在训练看起来像这样的文本的模型时，可能会出错的地方是什么？相反，模型可能从这个页面中提取哪些有用的信息？
交付物：3-4 句回应。

(c) 什么使一个好的训练示例具有高度的上下文性。描述一个应用领域，在这个领域中，这个示例可能对训练数据有用，以及一个它可能没有用的地方。
交付物：1-2 句回应。

(d) 让我们看更多的例子，以便更好地了解 Common Crawl 中的内容。浏览另外 25 个 WET 记录。对于每条记录，非常简要地评论文档的语言（如果可以识别），域名，页面类型等。你需要看到多少个例子才能确定你认为的“高质量”网页？
交付物：对 25 个文档进行简短注释，包括文档的语言、域名、页面类型和其他任何关于文档的杂项注释。直到你看到一个高质量的示例。示例的数量取决于你看到高质量示例的时间。


运行以下命令来下载数据集
```bash
wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-18/segments/1744889135610.12/warc/CC-MAIN-20250417135010-20250417165010-00065.warc.gz

wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-18/segments/1744889135610.12/wet/CC-MAIN-20250417135010-20250417165010-00065.warc.wet.gz
```

In [22]:
!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-18/segments/1744889135610.12/warc/CC-MAIN-20250417135010-20250417165010-00065.warc.gz


--2026-01-23 22:19:36--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-18/segments/1744889135610.12/warc/CC-MAIN-20250417135010-20250417165010-00065.warc.gz
Resolving data.commoncrawl.org (data.commoncrawl.org)... 18.238.238.35, 18.238.238.39, 18.238.238.18, ...
Connecting to data.commoncrawl.org (data.commoncrawl.org)|18.238.238.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1121135968 (1.0G) [application/octet-stream]
Saving to: ‘CC-MAIN-20250417135010-20250417165010-00065.warc.gz.1’

CC-MAIN-20250417135   0%[                    ]   5.47M  --.-KB/s    in 11m 19s 

2026-01-23 22:31:45 (8.24 KB/s) - Read error at byte 5733382/1121135968 (Connection reset by peer). Retrying.

--2026-01-23 22:31:46--  (try: 2)  https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-18/segments/1744889135610.12/warc/CC-MAIN-20250417135010-20250417165010-00065.warc.gz
Connecting to data.commoncrawl.org (data.commoncrawl.org)|18.238.238.35|:443... connected.
HTTP request s

In [ ]:

!wget https://data.commoncrawl.org/crawl-data/CC-MAIN-2025-18/segments/1744889135610.12/wet/CC-MAIN-20250417135010-20250417165010-00065.warc.wet.gz

我们可以使用
```bash
zcat CC-MAIN-20250417135010-20250417165010-00065.warc.gz | less

zcat CC-MAIN-20250417135010-20250417165010-00065.warc.wet.gz | less
```
在命令行中进行预览，less表示可以使用键盘方向键、Page Up 和 Page Down 来浏览文件。要退出，请按“q“。


`.warc.gz` 和 `.warc.wet.gz` 都是 **WARC（Web ARChive）格式** 的压缩文件，广泛用于大规模网页抓取数据（如 Common Crawl）。

## 一、`.warc.gz` 与 `.warc.wet.gz` 的区别

### 1. `.warc.gz`

**原始网页归档文件**，包含完整抓取结果：HTTP 请求 / 响应头、HTML 原文、二进制资源（图片、PDF 等）、元数据（抓取时间、URL、状态码等）。

### 2. `.warc.wet.gz`

**WET = Web Extracted Text**

从 `.warc.gz` 中**抽取后的纯文本**，仅保留网页的可读文本内容，已去除 HTML 标签、脚本、样式等

适合NLP 训练（LM、Embedding、Topic），文本挖掘，语料构建。




In [ ]:
!gunzip -c CC-MAIN-20250417135010-20250417165010-00065.warc.gz > data/CC_data.warc
!gunzip -c CC-MAIN-20250417135010-20250417165010-00065.warc.wet.gz > data/CC_data.warc.wet


In [23]:
# 从 warcio 库中导入 ArchiveIterator
# ArchiveIterator 用于按“记录（record）”顺序流式读取 WARC 文件
from warcio.archiveiterator import ArchiveIterator

# 导入 gzip，用于读取 .gz 压缩格式的 WARC 文件
import gzip

# 导入 html 模块，用于将 HTML 实体（如 &#x4EBA;）解码为正常字符
import html 

# WARC 文件路径（Common Crawl 的一个分片）
# 注意：变量名 flie_path 拼写不影响运行，但语义上建议改为 file_path
flie_path  = 'CC-MAIN-20250417135010-20250417165010-00065.warc.gz'

# 以二进制只读模式打开 gzip 压缩的 WARC 文件
# 必须使用 "rb"，因为 WARC 是二进制格式
with gzip.open(flie_path, "rb") as f:

    # 使用 ArchiveIterator 对 WARC 文件进行流式遍历
    # 每次循环返回一个 WARC record（请求、响应、元数据等）
    for step,record in enumerate(ArchiveIterator(f)):
       
        # 只处理类型为 "response" 的记录
        # response 表示真实的 HTTP 响应内容（网页、图片等）
        if record.rec_type == "response" and step > 5:

            # 从 WARC 头中获取该网页对应的原始 URL
            url = record.rec_headers.get_header("WARC-Target-URI")

            # 读取 HTTP 响应体内容（网页原始字节流）
            # decode 为 utf-8，忽略非法字符，防止程序崩溃
            html1 = record.content_stream().read().decode("utf-8", errors="ignore")

            # 简单判断该响应是否为 HTML 页面
            # 这是一个经验性过滤，避免处理图片、PDF 等非 HTML 内容
            if "<html" in html1.lower():

                # 输出网页 URL
                print(url)

                # 对 HTML 内容前 500 个字符进行：
                # 1. 截断（避免输出过多内容）
                # 2. HTML 实体解码（&#x4EBA; → 人）
                print(html.unescape(html1[:500]))

                # 找到第一个 HTML 页面后立即退出循环
            
                break


http://13.usnccm.org/
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"
  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" version="XHTML+RDFa 1.0" dir="ltr"
  xmlns:content="http://purl.org/rss/1.0/modules/content/"
  xmlns:dc="http://purl.org/dc/terms/"
  xmlns:foaf="http://xmlns.com/foaf/0.1/"
  xmlns:og="http://ogp.me/ns#"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:sioc="http://rdfs.org/sioc/ns#"
  xmlns:sioct="http://rdfs.org/sioc/ty


In [24]:
# 从 warcio 库中导入 ArchiveIterator
# ArchiveIterator 用于按顺序流式遍历 WARC / WET 文件中的每一条记录
from warcio.archiveiterator import ArchiveIterator

# 导入 gzip 模块，用于读取 .gz 压缩格式的文件
import gzip

# 指定 Common Crawl 的 WET 文件路径
# .warc.wet.gz 文件中存放的是已抽取好的“纯文本内容”
flie_path  = 'CC-MAIN-20250417135010-20250417165010-00065.warc.wet.gz'

# 以二进制只读模式打开 gzip 压缩的 WET 文件
# 必须使用 "rb"，因为 WARC/WET 属于二进制流格式
with gzip.open(flie_path, "rb") as f:

    # 使用 ArchiveIterator 对 WET 文件进行流式迭代
    # 每次循环返回一个 WARC record（一条网页对应一条 record）
    for record in ArchiveIterator(f):

        # 只处理类型为 "conversion" 的记录
        # conversion 表示从 HTML 中抽取出的“可读纯文本”
        if record.rec_type == "conversion":

            # 从 WARC 头信息中获取该文本对应的原始网页 URL
            url = record.rec_headers.get_header("WARC-Target-URI")

            # 读取当前 record 的正文内容（纯文本字节流）
            # 解码为 UTF-8 字符串，忽略非法字符，防止解码异常
            text = record.content_stream().read().decode("utf-8", errors="ignore")

            # 判断文本在去除首尾空白字符后是否仍然有内容
            # 用于过滤空页面或无有效文本的网页
            if len(text.strip()) > 0:

                # 打印网页对应的 URL
                print(url)

                # 打印正文内容的前 400 - 500 个字符
                # 用于快速查看文本样本，避免一次性输出过多内容
                print(text[400:500])

                # 找到第一条有效文本记录后立即退出循环
                # 适用于“快速验证 WET 文件内容是否正常”的场景
                break


http://0371rykj.com/ipfhsb/34.html
> ip防護(hù)系列 >
產(chǎn)品詳情/ products details
恒溫恒濕試驗(yàn)箱
產(chǎn)品用途
恒溫恒濕試驗(yàn)箱是航空、汽車(chē)、家電、科研等領(


# 作业二

**问题（提取文本）：3分**

（a）编写一个函数，用于从包含原始 HTML 的字节字符串中提取文本。使用 `resiliparse.extract.html2text.extract_plain_text` 执行提取操作。该函数需要一个字符串作为输入，因此您先需要将字节字符串解码为 Unicode 字符串。请注意，输入的字节字符串可能并未采用 UTF-8 编码，因此您的函数应具备检测编码的能力，以防 UTF-8 解码失败。此外，Resiliparse 还提供 `resiliparse.parse.encoding.detect_encoding()`，这可能会有用。

**交付物**：一个函数，接收包含 HTML 的字节字符串，并返回提取出的文本字符串。实现适配器 `[run_extract_text_from_html_bytes]`，并确保其通过 uv 运行
`pytest -k test_extract_text_from_html_bytes`

（b）对单个 WARC 文件运行你的文本提取功能。将其输出与提取的内容进行比较，对应 WET 文件中的文本。你注意到哪些差异和/或相似之处？哪种提取方式似乎更好？

**可交付成果**：用您自定义函数提取的文本与 WET 文件中的提取文本进行比较和对比，撰写 2–3 句话的回应。

---


In [3]:
%%writefile run_extract_text_from_html_bytes.py

from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.encoding import detect_encoding


def run_extract_text_from_html_bytes(html_bytes: bytes) -> str:
    """
    从包含 HTML 的字节字符串中提取纯文本。

    参数:
        html_bytes (bytes): 原始 HTML 字节串

    返回:
        str: 提取出的纯文本
    """
    # 1. 尝试直接用 UTF-8 解码（最快路径）
    try:
        html_text = html_bytes.decode("utf-8")
    except UnicodeDecodeError:
        # 2. UTF-8 失败时，使用 resiliparse 的编码检测
        encoding = detect_encoding(html_bytes)
        # 使用检测到的编码进行解码，忽略无法解码的字符
        html_text = html_bytes.decode(encoding, errors="ignore")

    # 3. 使用 Resiliparse 从 HTML 字符串中提取纯文本
    text = extract_plain_text(html_text)

    return text


Writing run_extract_text_from_html_bytes.py


In [26]:
import gzip
from warcio.archiveiterator import ArchiveIterator
file_path = "CC-MAIN-20250417135010-20250417165010-00065.warc.gz"
with gzip.open(file_path, "rb") as f:
    for record in ArchiveIterator(f):
        if record.rec_type == "response":
            # 只处理 HTML 页面
            content_type = record.http_headers.get_header("Content-Type", "")
            if "text/html" not in content_type:
                continue

            # 1. 读取网页的原始 HTML（bytes）
            html_bytes = record.content_stream().read()

            # 2. 使用函数提取文本
            text = run_extract_text_from_html_bytes(html_bytes)

            # 3. 查看结果
            if text.strip():
                print(record.rec_headers.get_header("WARC-Target-URI"))
                print(text[:500])
                break


http://0371rykj.com/ipfhsb/34.html
久久久久女人精品毛片,99久久精品无码一区二区毛片,被老外的又粗又大日出了水,一边吃奶一边哭乱抻又乱扭

        • <th id="gckmo"></th>
        • <ul id="gckmo"><center id="gckmo"></center></ul>
      •  
  
 
 
 
         
   
         
    
         
    
        恒溫恒濕試驗(yàn)箱
        
	在線(xiàn)咨詢(xún)
    
         
    
         
      淋雨試驗(yàn)箱 
     
        上海林頻儀器股份有限公司Shanghai Linpin Instrument Stock Co Ltd
         
     
        服務(wù)熱線(xiàn)：4000 662 888 
         手機(jī)咨詢(xún)：13818467052
        
    
         
    
         
     



# 作业三

问题（语言识别）：6 分

(a) 编写一个函数，该函数将接收一个 Unicode 字符串，并识别其中出现的主要语言。你的函数应返回一个元组，包含该语言的标识符以及一个介于 0 和 1 之间的分数，表示对该预测的置信度。

**交付物：** 一个执行语言识别的函数，返回其最可能的语言预测及相应得分。实现适配器 `[run_identify_language]`，并确保它能通过 uv 中运行 `pytest -k test_identify_language` 的两项测试。请注意，这些测试假定英文使用特定的字符串标识符（“en”），中文使用 “zh”，因此你的测试适配器应根据需要执行任何必要的重新映射操作。

---

(b) 语言模型在推理阶段的行为在很大程度上取决于其训练所用的数据。因此，数据过滤管道中的问题可能会导致下游出现一系列难题。你认为语言识别环节出现问题可能引发哪些具体问题？在风险更高的场景中（例如部署面向用户的产品时），你将如何有效应对并缓解这些问题？

**可交付成果：** 2 至 5 句话的回复。

---

(c) 将您的语言识别系统应用于从 WARC 文件中提取的文本（通过您先前实现的文本提取功能）。在 20 个随机示例中手动识别语言，并将您的标注与分类器的预测结果进行对比。报告任何分类错误。有多少比例的文档是英语？根据您的观察，使用何种合适的分类器置信度阈值进行过滤会比较有效？

**交付物：** 2 至 5 句话的回复。





网络上包含着用数千种语言编写的页面。然而，在大多数计算预算下，训练一种能够有效利用如此多样化数据并实现大规模应用的多语言模型，仍是一项极具挑战性的任务。因此，许多源自Common Crawl的数据集所涵盖的语言种类相对有限。
一个适用于此目的的实用库是 fastText https://fasttext.cc，它提供了高效的文本分类器。该库既支持基于您自己的数据训练分类器的基础设施，也内置了一系列预训练模型，其中包括用于语言识别的模型。它提供了高效的文本分类器。该库既支持基于您自己的数据训练分类器的基础设施，也内置了一系列预训练模型，其中包括用于语言识别的模型。您可以从https://fasttext.cc/docs/en/language-identification.html 下载 fastText 语言识别模型 

通常，语言过滤器会利用分类器给出的得分来决定是否保留某页面。请使用fastText语言识别分类器实现一个语言识别过滤器，该过滤器应能输出一个非负分数，以表示其预测的置信度。


In [10]:
# 下载模型 速度更快，准确度也略高，但文件大小为 126MB

!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin


--2026-01-26 12:22:57--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘lid.176.bin’

lid.176.bin         100%[===================>] 125.18M  9.82MB/s    in 11s     

2026-01-26 12:23:08 (11.3 MB/s) - ‘lid.176.bin’ saved [131266198/131266198]



In [11]:
# 下载模型 是该模型的压缩版本，文件大小为 917kB。

!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

--2026-01-26 12:23:08--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
Connecting to 127.0.0.1:7890... connected.
Proxy request sent, awaiting response... 200 OK
Length: 938013 (916K) [binary/octet-stream]
Saving to: ‘lid.176.ftz’

lid.176.ftz         100%[===================>] 916.03K  4.73MB/s    in 0.2s    

2026-01-26 12:23:09 (4.73 MB/s) - ‘lid.176.ftz’ saved [938013/938013]



In [12]:
import fasttext

# 禁用 mmap
model = fasttext.load_model("lid.176.bin", mmap=False)
print(model.predict("This is a test."))

: 

In [4]:
%%writefile run_identify_language.py
import fasttext
from functools import lru_cache

# fastText 语言识别模型路径
MODEL_PATH = "lid.176.bin"


@lru_cache(maxsize=1)
def _load_model():
    """
    加载 fastText 语言识别模型（单例模式）

    使用 lru_cache 的原因：
    - fastText 模型体积较大（~126MB）
    - 避免在每次函数调用时重复加载模型
    - 提高整体运行效率，尤其在批量文本处理时
    """
    return fasttext.load_model(MODEL_PATH)


def run_identify_language(text: str):
    """
    识别输入 Unicode 字符串中的主要语言

    参数：
        text (str): 任意 Unicode 文本字符串

    返回：
        (language_id, confidence_score)
        - language_id: 语言标识符（如 "en", "zh"）
        - confidence_score: 介于 [0, 1] 的置信度分数

    注意：
    - 该函数是测试要求的“适配器函数”
    - 必须保证英文返回 "en"，中文返回 "zh"
    """

    # 对空字符串或只包含空白符的情况进行兜底处理
    # fastText 在这种情况下预测结果不可靠
    if not text or not text.strip():
        return ("unknown", 0.0)

    # 加载（或从缓存中获取）语言识别模型
    model = _load_model()

    # fastText 的 predict 接口：
    # - 返回 labels 和 scores 两个列表
    # - labels 形如 "__label__en"
    # - scores 为预测置信度
    labels, scores = model.predict(
        text.replace("\n", " "),  # 去除换行，避免影响模型判断
        k=1                        # 只取概率最高的一个语言
    )

    # 取出预测结果
    raw_label = labels[0]                 # "__label__en"
    confidence = float(scores[0])         # 置信度（0~1）

    # 去掉 fastText 特有的 "__label__" 前缀
    language = raw_label.replace("__label__", "")

    # 根据作业测试要求，对语言标签进行统一映射
    # fastText 可能返回 zh, zh-cn, zh-tw 等
    if language.startswith("zh"):
        language = "zh"
    elif language == "en":
        language = "en"
    # 其他语言保持原样（如 "fr", "de", "ja" 等）

    return (language, confidence)

import fasttext
from functools import lru_cache

# fastText 语言识别模型路径
# 在 Together 集群或课程环境中通常已预置在该位置
MODEL_PATH = "/data/classifiers/lid.176.bin"


@lru_cache(maxsize=1)
def _load_model():
    """
    加载 fastText 语言识别模型（单例模式）

    使用 lru_cache 的原因：
    - fastText 模型体积较大（~126MB）
    - 避免在每次函数调用时重复加载模型
    - 提高整体运行效率，尤其在批量文本处理时
    """
    return fasttext.load_model(MODEL_PATH)


def run_identify_language(text: str):
    """
    识别输入 Unicode 字符串中的主要语言

    参数：
        text (str): 任意 Unicode 文本字符串

    返回：
        (language_id, confidence_score)
        - language_id: 语言标识符（如 "en", "zh"）
        - confidence_score: 介于 [0, 1] 的置信度分数

    注意：
    - 该函数是测试要求的“适配器函数”
    - 必须保证英文返回 "en"，中文返回 "zh"
    """

    # 对空字符串或只包含空白符的情况进行兜底处理
    # fastText 在这种情况下预测结果不可靠
    if not text or not text.strip():
        return ("unknown", 0.0)

    # 加载（或从缓存中获取）语言识别模型
    model = _load_model()

    # fastText 的 predict 接口：
    # - 返回 labels 和 scores 两个列表
    # - labels 形如 "__label__en"
    # - scores 为预测置信度
    labels, scores = model.predict(
        text.replace("\n", " "),  # 去除换行，避免影响模型判断
        k=1                        # 只取概率最高的一个语言
    )

    # 取出预测结果
    raw_label = labels[0]                 # "__label__en"
    confidence = float(scores[0])         # 置信度（0~1）

    # 去掉 fastText 特有的 "__label__" 前缀
    language = raw_label.replace("__label__", "")

    # 根据作业测试要求，对语言标签进行统一映射
    # fastText 可能返回 zh, zh-cn, zh-tw 等
    if language.startswith("zh"):
        language = "zh"
    elif language == "en":
        language = "en"
    # 其他语言保持原样（如 "fr", "de", "ja" 等）

    return (language, confidence)

if __name__ == "__main__":
    examples = [
        "This is a simple English sentence.",
        "这是一个用于测试的中文句子。",
        "Bonjour, comment allez-vous ?",
        "こんにちは、元気ですか？",
        "",
        "    "
    ]

    for text in examples:
        lang, score = run_identify_language(text)
        print(f"文本: {repr(text)}")
        print(f"预测语言: {lang}, 置信度: {score:.4f}")
        print("-" * 50)



Overwriting run_identify_language.py


In [8]:
!python run_identify_language.py

# 作业四

## 问题（屏蔽 PII）：3 分

### 1. 屏蔽电子邮件地址

编写一个函数来屏蔽电子邮件。该函数将接收一个字符串作为输入，并用字符串
**`"|||EMAIL_ADDRESS|||"`**
替换所有出现的电子邮件地址。要检测电子邮件地址，你可以查阅能够可靠完成此任务的正则表达式。

**可交付成果：**
一个函数，用于将给定字符串中的所有电子邮件地址替换为字符串 `"|||EMAIL_ADDRESS|||"`，并返回一个元组，其中包含新字符串以及被屏蔽的实例数量。实现适配器 **`[run_mask_emails]`**，并确保其通过
`uv run pytest -k test_mask_emails` 中的所有测试。

---

### 2. 屏蔽电话号码

编写一个函数来屏蔽电话号码。该函数将接收一个字符串作为输入，并用字符串
**`"|||PHONE_NUMBER|||"`**
替换所有出现的电话号码。要可靠地实现这一点可能极具挑战性，因为电话号码的书写形式可能极其多样。

你应尽量捕获**美国最常用的电话号码格式**，并能有效应对轻微的语法偏差。

**可交付成果：**
一个函数，用于将给定字符串中的电话号码替换为字符串 `"|||PHONE_NUMBER|||"`，并返回包含新字符串及被屏蔽的电话号码数量的元组。实现适配器 **`[run_mask_phone_numbers]`**，并确保其通过
`uv run pytest -k test_mask_phones`。

---

### 3. 屏蔽 IP 地址

编写一个函数来屏蔽 IP 地址。对于这个问题，只需关注 **IPv4 地址**（由四个介于 0 到 255 之间的数字组成，各数字之间用点分隔）。

你的函数将接收一个字符串作为输入，并将所有出现的 IP 地址替换为字符串
**`"|||IP_ADDRESS|||"`**。

**可交付成果：**
一个函数，用于将给定字符串中的 IPv4 地址替换为字符串 `"|||IP_ADDRESS|||"`，并返回一个元组，其中包含新字符串以及被掩码替换的实例数量。实现适配器 **`[run_mask_ips]`**，并确保其通过
`uv run pytest -k test_mask_ips`。

---

### 4. 分析题（2–5 句）

你认为在训练集中如果**天真地**应用这些过滤器，可能会在语言模型的下游环节引发哪些问题？你又该如何缓解这些问题？

**交付物：** 2 至 5 句话的回复。

---

### 5. 实验分析（2–5 句）

在从 WARC 文件中提取的文本上运行你的 PII 脱敏函数（通过你先前实现的文本提取功能）。随机选取 20 个已进行替换的示例进行检查，并列举一些**假阳性（false positives）**和**假阴性（false negatives）**的具体示例。

**交付物：** 2 至 5 句话的回复。




In [ ]:
import re
from typing import Tuple

def run_mask_emails(text: str) -> Tuple[str, int]:
    """
    屏蔽字符串中的电子邮件地址。

    参数:
        text (str): 输入文本

    返回:
        (masked_text, count)
    """

    # 常见电子邮件地址的正则表达式
    email_pattern = re.compile(
        r'\b[a-zA-Z0-9._%+-]+'
        r'@'
        r'[a-zA-Z0-9.-]+'
        r'\.[a-zA-Z]{2,}\b'
    )

    # findall 用于统计匹配数量
    matches = email_pattern.findall(text)

    # sub 用于统一替换
    masked_text = email_pattern.sub("|||EMAIL_ADDRESS|||", text)

    return masked_text, len(matches)



Writing mask_emails.py


In [ ]:

import re
from typing import Tuple
def run_mask_phone_numbers(text: str) -> Tuple[str, int]:


    phone_pattern = re.compile(
        r'''
        \b
        (?:\+?1[\s.-]?)?          # 可选国家码
        (?:\(?\d{3}\)?[\s.-]?)    # 区号
        \d{3}[\s.-]?\d{4}         # 主号码
        \b
        ''',
        re.VERBOSE
    )

    matches = phone_pattern.findall(text)
    masked_text = phone_pattern.sub("|||PHONE_NUMBER|||", text)

    return masked_text, len(matches)


Overwriting mask_phone_numbers.py


In [ ]:
import re
from typing import Tuple

def run_mask_ips(text: str) -> Tuple[str, int]:
    """
    屏蔽 IPv4 地址
    """

    ip_pattern = re.compile(
        r'\b'
        r'(?:25[0-5]|2[0-4]\d|1?\d{1,2})'
        r'(?:\.'
        r'(?:25[0-5]|2[0-4]\d|1?\d{1,2})){3}'
        r'\b'
    )

    matches = ip_pattern.findall(text)
    masked_text = ip_pattern.sub("|||IP_ADDRESS|||", text)

    return masked_text, len(matches)



Overwriting mask_ips.py
